In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [4]:
# Setup Selenium Chrome driver (headless mode)
def setup_driver():
    options = Options()
    options.add_argument('--headless')  # Runs Chrome in headless mode
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    service = Service('/opt/homebrew/bin/chromedriver')  # Specify path to your ChromeDriver
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [8]:
# Initialize WebDriver
driver = setup_driver()
# Open Ground News
url = "https://ground.news"
driver.get(url)
time.sleep(5)  # Allow the page to load


In [10]:

# Scroll to load all categories
scroll_pause_time = 3
num_scrolls = 5  # Adjust as needed

for _ in range(num_scrolls):
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)  # Scroll to the bottom
    time.sleep(scroll_pause_time)



In [12]:
# Parse the page source
soup = BeautifulSoup(driver.page_source, "html.parser")

# Extract article links
article_links = []
for a_tag in soup.find_all("a", href=True):
    href = a_tag["href"]
    if "/daily-briefing/" in href and not href.startswith("#"):  # Filter for article links
        full_link = href if href.startswith("http") else f"https://ground.news{href}"
        if full_link not in article_links:  # Avoid duplicates
            article_links.append(full_link)

# Print all collected links
if article_links:
    print("News Links Found:")
    for link in article_links:
        print(link)
else:
    print("No news links found. Please check the structure of the website.")

# Close the browser
driver.quit()

No news links found. Please check the structure of the website.


In [ ]:
# Print all collected links
print("News Links Found:")
for link in article_links:
    print(link)

# Close the browser
driver.quit()

In [ ]:
# Scroll and Load Articles
scroll_pause_time = 3
num_scrolls = 15  # Adjust the number of scrolls as needed



In [ ]:

# Visit each category link and scrape news items
for category_link in category_links:
    driver.get(category_link)
    time.sleep(5)  # Allow category page to load

    # Scroll to load more news articles on the category page
    for _ in range(5):  # Adjust number of scrolls per category
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(scroll_pause_time)

    # Parse the category page for news articles
    category_soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = category_soup.find_all("div", class_="card-container")  # Adjust class to match news items

    for article in articles:
        try:
            title = article.find("h3").get_text(strip=True)  # Adjust tag for titles
            link = article.find("a", href=True)["href"]  # Extract article link
            if not link.startswith("http"):
                link = f"https://ground.news{link}"
            sentiment_score = random.randint(1, 5)  # Assign random sentiment score
            all_news.append({"title": title, "link": link, "sentiment_score": sentiment_score})
        except Exception as e:
            print(f"Error extracting news article: {e}")

driver.quit()


In [ ]:
# Save data to CSV
if all_news:
    df = pd.DataFrame(all_news)
    df.to_csv("ground_news_actual_articles.csv", index=False)
    print(f"Saved {len(all_news)} articles to ground_news_actual_articles.csv")
else:
    print("No articles found. Please check the structure of the website.")